# 자연어처리 + 분류
### 자연어 기반 기후기술분류 AI 경진대회 데이터 사용

# CNN 문장 분류

## 1. 데이터 & 라이브러리 준비

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import json
import os
from tqdm import tqdm
from ast import literal_eval
from konlpy.tag import Okt

import sklearn
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import log_loss, accuracy_score,f1_score
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
#from transformers import BertTokenizer, BertModel

#os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
data_path = 'data/open/'

train=pd.read_csv(data_path + 'train.csv')
test=pd.read_csv(data_path +'test.csv')
sample_submission=pd.read_csv(data_path +'sample_submission.csv')

In [3]:
train=train[['과제명', '요약문_연구내용','label']]
test=test[['과제명', '요약문_연구내용']]

train['요약문_연구내용'].fillna('NAN', inplace=True)
test['요약문_연구내용'].fillna('NAN', inplace=True)

In [4]:
train['data']=train['과제명']+train['요약문_연구내용']
test['data']=test['과제명']+test['요약문_연구내용']

In [5]:
df = train.copy()

# 데이터 용량이 매우 커 두 개로 나눔
df1 = df[:100000]
df2 = df[100000:]
df1

,과제명,요약문_연구내용,label,data
0,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,(가) 외래 및 돌발해충의 발생조사 및 종 동정\n\n\n ○ 대상해충 : 최...,24,유전정보를 활용한 새로운 해충 분류군 동정기술 개발(가) 외래 및 돌발해충의 발생조...
1,대장암의 TRAIL 내성 표적 인자 발굴 및 TRAIL 반응 예측 유전자 지도 구축...,1차년도\n1) Microarray를 통한 선천적 TRAIL 내성 표적 후보 유전자...,0,대장암의 TRAIL 내성 표적 인자 발굴 및 TRAIL 반응 예측 유전자 지도 구축...
2,비목질계 셀룰로오스 식물자원을 활용한 기능성 부직포 및 고부가가치 뷰티케어 faci...,* 식물계자원 정련 및 최적 신서란 파이버 기초연구 개발\n\n\n- Tencel/...,0,비목질계 셀룰로오스 식물자원을 활용한 기능성 부직포 및 고부가가치 뷰티케어 faci...
3,소화기 암 진단용 분자영상 형광프로브 개발,# 소화기 암 진단용 분자영상 형광프로브 개발\n\n\n- 국소 도포형 소화기 암 ...,0,소화기 암 진단용 분자영상 형광프로브 개발# 소화기 암 진단용 분자영상 형광프로브 ...
4,위암환자의 항암제반응예측을 위한 BRCA 발현검사,-In situ hybridization 검사의 정확성을 확인하기 위해 위암세포주 ...,0,위암환자의 항암제반응예측을 위한 BRCA 발현검사-In situ hybridizat...
...,...,...,...,...
99995,"재배안정성, 복합내병성 토마토 품종육성","- 토마토 유전자원의 탐색, 수집, 평가하여 육종소재 활용\n\n\n - 양친으로...",23,"재배안정성, 복합내병성 토마토 품종육성- 토마토 유전자원의 탐색, 수집, 평가하여 ..."
99996,가상현실 바람효과를 재현하는 에어블로워 클러스터링 기술 개발,&lt;주관기관 개발내용&gt;\n\n\n？ 가상현실 연동용 윈드블로워 개발\n\n...,0,가상현실 바람효과를 재현하는 에어블로워 클러스터링 기술 개발&lt;주관기관 개발내용...
99997,혈관재협착을 일으키는 평활근전구세포의 REDOX 반응 연구,1차년도 혈관전구세포의 분화에 미치는 redox 단백질의 기능 연구\n\n1) iP...,0,혈관재협착을 일으키는 평활근전구세포의 REDOX 반응 연구1차년도 혈관전구세포의 분...
99998,갈색날개매미충의 신호화학물질 개발 및 이용방법 연구,○ 외래해충 유인 및 기피물질 선발\n\n\n ○ 선발된 유인물질과 기피물질의 작용...,24,갈색날개매미충의 신호화학물질 개발 및 이용방법 연구○ 외래해충 유인 및 기피물질 선...


## 2. 텍스트 전처리

In [6]:
okt = Okt()

def text_preprocess(x):
    text=[]
    a = re.sub('[^가-힣0-9a-zA-Z\\s]', '',x)
    for j in a.split():
        text.append(j)
    return ' '.join(text)

def tokenize(x):
    text = []
    tokens = okt.pos(x)
    for token in tokens :
        if token[1] == 'Adjective' or token[1]=='Adverb' or token[1] == 'Determiner' or token[1] == 'Noun' or token[1] == 'Verb' or 'Unknown':
            text.append(token[0])
    return text

In [7]:
tqdm.pandas()
df1['contents'] = df1['data'].apply(lambda x : text_preprocess(x))
df1['contents'] = df1['contents'].progress_apply(lambda x: tokenize(x))
df1

c:\users\samsung\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
100%|████████████████████████████████████████████████████████████████████████| 100000/100000 [1:40:42<00:00, 16.55it/s]
c:\users\samsung\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,과제명,요약문_연구내용,label,data,contents
0,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,(가) 외래 및 돌발해충의 발생조사 및 종 동정\n\n\n ○ 대상해충 : 최...,24,유전정보를 활용한 새로운 해충 분류군 동정기술 개발(가) 외래 및 돌발해충의 발생조...,"[유전, 정보, 를, 활용, 한, 새로운, 해충, 분류군, 동정, 기술, 개발, 가..."
1,대장암의 TRAIL 내성 표적 인자 발굴 및 TRAIL 반응 예측 유전자 지도 구축...,1차년도\n1) Microarray를 통한 선천적 TRAIL 내성 표적 후보 유전자...,0,대장암의 TRAIL 내성 표적 인자 발굴 및 TRAIL 반응 예측 유전자 지도 구축...,"[대장암, 의, TRAIL, 내, 성, 표적, 인자, 발굴, 및, TRAIL, 반응..."
2,비목질계 셀룰로오스 식물자원을 활용한 기능성 부직포 및 고부가가치 뷰티케어 faci...,* 식물계자원 정련 및 최적 신서란 파이버 기초연구 개발\n\n\n- Tencel/...,0,비목질계 셀룰로오스 식물자원을 활용한 기능성 부직포 및 고부가가치 뷰티케어 faci...,"[비목, 질계, 셀룰로오스, 식물, 자원, 을, 활용, 한, 기능, 성, 부직포, ..."
3,소화기 암 진단용 분자영상 형광프로브 개발,# 소화기 암 진단용 분자영상 형광프로브 개발\n\n\n- 국소 도포형 소화기 암 ...,0,소화기 암 진단용 분자영상 형광프로브 개발# 소화기 암 진단용 분자영상 형광프로브 ...,"[소화기, 암, 진단, 용, 분자영상, 형광, 프로, 브, 개발, 소화기, 암, 진..."
4,위암환자의 항암제반응예측을 위한 BRCA 발현검사,-In situ hybridization 검사의 정확성을 확인하기 위해 위암세포주 ...,0,위암환자의 항암제반응예측을 위한 BRCA 발현검사-In situ hybridizat...,"[위암, 환자, 의, 항암제, 반응, 예측, 을, 위, 한, BRCA, 발현, 검사..."
...,...,...,...,...,...
99995,"재배안정성, 복합내병성 토마토 품종육성","- 토마토 유전자원의 탐색, 수집, 평가하여 육종소재 활용\n\n\n - 양친으로...",23,"재배안정성, 복합내병성 토마토 품종육성- 토마토 유전자원의 탐색, 수집, 평가하여 ...","[재배, 안정, 성, 복합, 내, 병, 성, 토마토, 품종, 육성, 토마토, 유전자..."
99996,가상현실 바람효과를 재현하는 에어블로워 클러스터링 기술 개발,&lt;주관기관 개발내용&gt;\n\n\n？ 가상현실 연동용 윈드블로워 개발\n\n...,0,가상현실 바람효과를 재현하는 에어블로워 클러스터링 기술 개발&lt;주관기관 개발내용...,"[가상현실, 바람, 효과, 를, 재현, 하는, 에어, 블, 로, 워, 클러스터링, ..."
99997,혈관재협착을 일으키는 평활근전구세포의 REDOX 반응 연구,1차년도 혈관전구세포의 분화에 미치는 redox 단백질의 기능 연구\n\n1) iP...,0,혈관재협착을 일으키는 평활근전구세포의 REDOX 반응 연구1차년도 혈관전구세포의 분...,"[혈관, 재협착, 을, 일으키는, 평활, 근, 전, 구, 세포, 의, REDOX, ..."
99998,갈색날개매미충의 신호화학물질 개발 및 이용방법 연구,○ 외래해충 유인 및 기피물질 선발\n\n\n ○ 선발된 유인물질과 기피물질의 작용...,24,갈색날개매미충의 신호화학물질 개발 및 이용방법 연구○ 외래해충 유인 및 기피물질 선...,"[갈색, 날개, 매미, 충의, 신호, 화학물질, 개발, 및, 이용, 방법, 연구, ..."


In [7]:
tqdm.pandas()
df2['contents'] = df2['data'].apply(lambda x : text_preprocess(x))
df2['contents'] = df2['contents'].progress_apply(lambda x: tokenize(x))
df2

c:\users\samsung\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
100%|████████████████████████████████████████████████████████████████████████████| 74304/74304 [55:24<00:00, 22.35it/s]
c:\users\samsung\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,과제명,요약문_연구내용,label,data,contents
100000,생산성 향상을 위한 천연광물 추출물을 이용한 양돈용 액상 단미사료의 개발,？천연 광물 추출물 양돈 전용 미네랄 단미사료 개발\n\n\n - 추출 광물질(세리...,0,생산성 향상을 위한 천연광물 추출물을 이용한 양돈용 액상 단미사료의 개발？천연 광물...,"[생산, 성, 향상, 을, 위, 한, 천연, 광물, 추출, 물, 을, 이용, 한, ..."
100001,수출단감 생산을 위한 관비재배 기술 개발,(시험1) 수출단감 관비재배 가능성 구명\n\n\n 시험품종 : ‘부유’\n\n...,0,수출단감 생산을 위한 관비재배 기술 개발(시험1) 수출단감 관비재배 가능성 구명\n...,"[수출, 단감, 생산, 을, 위, 한, 관비, 재배, 기술, 개발, 시험, 1, 수..."
100002,환자 안전 및 진료 효과성 제고를 위한 프로세스 지능화 및 진료 의사결정지원 기술 개발,1단계 연구 목표 및 내용 - 의료기관 프로세스 관리 운영 최적화 기술 개발\n\n...,0,환자 안전 및 진료 효과성 제고를 위한 프로세스 지능화 및 진료 의사결정지원 기술 ...,"[환자, 안전, 및, 진료, 효과, 성, 제, 고, 를, 위, 한, 프로세스, 지능..."
100003,"식품자원의 비타민 A, E 및 K 성분 DB 구축",□ 시료를 분석하기 전에 표준물질 (CRM 또는 SRM)을 이용하여 각 분석항목별 ...,0,"식품자원의 비타민 A, E 및 K 성분 DB 구축□ 시료를 분석하기 전에 표준물질 ...","[식품, 자원, 의, 비타민, A, E, 및, K, 성분, DB, 구축, 시료, 를..."
100004,현장계측 진동모드해석을 통한 케이슨 방파제의 구조성능 평가 기술,◦ 본 연구는 2년의 연구기간으로서 각 연차별 주요 연구내용은 다음과 같음.\n◦ ...,0,현장계측 진동모드해석을 통한 케이슨 방파제의 구조성능 평가 기술◦ 본 연구는 2년의...,"[현, 장계, 측, 진동, 모드, 해석, 을, 통한, 케이슨, 방파제, 의, 구조,..."
...,...,...,...,...,...
174299,혈관내막 증식을 치료하기 위한 표적분자 발굴,● 1차년: 혈관내막 증식과 관련된 표적분자 발굴 및 염증세포의 활성과 분화에 대한...,0,혈관내막 증식을 치료하기 위한 표적분자 발굴● 1차년: 혈관내막 증식과 관련된 표적...,"[혈관, 내막, 증식, 을, 치료, 하기, 위, 한, 표적, 분자, 발굴, 1, 차..."
174300,삼킴곤란 개선을 위한 물성조절 식품시스템에서의 유동층과립 복합소재의 유변물성 변화 ...,본 연구에서는 여러 가지 유변물성 측정기술을 이용하여 고령자의 삼킴곤란 개선을 위해...,0,삼킴곤란 개선을 위한 물성조절 식품시스템에서의 유동층과립 복합소재의 유변물성 변화 ...,"[삼킴, 곤란, 개선, 을, 위, 한, 물성, 조절, 식품, 시스템, 에서의, 유동..."
174301,식물 유전체 정보 고도화 및 핵심 유전자 군 진화연구,○ 공개된 식물 유전체 정보 및 대상 유전자 군 선발\n\n - 공개된 식물 유...,0,식물 유전체 정보 고도화 및 핵심 유전자 군 진화연구○ 공개된 식물 유전체 정보 및...,"[식물, 유전체, 정보, 고도화, 및, 핵심, 유전자, 군, 진화, 연구, 공개, ..."
174302,"콩 유전자원 내건성, 내습성 대량평가 체계구축 및 자원선발",[1차 년도(2019)] \n\n1) 내건 및 내습성 유전자원 수집\n\n○ 기존 ...,23,"콩 유전자원 내건성, 내습성 대량평가 체계구축 및 자원선발[1차 년도(2019)] ...","[콩, 유전자, 원, 내, 건성, 내, 습성, 대량, 평가, 체계, 구축, 및, 자..."


In [60]:
final_df = pd.concat([df1,df2,test]).reset_index(drop = True)[['contents','label']]
final_df.to_csv("final_df2.csv", index=False, encoding="utf-8-sig")
final_df

,contents,label
0,"['유전', '정보', '를', '활용', '한', '새로운', '해충', '분류군...",24.0
1,"['대장암', '의', 'TRAIL', '내', '성', '표적', '인자', '발...",0.0
2,"['비목', '질계', '셀룰로오스', '식물', '자원', '을', '활용', '...",0.0
3,"['소화기', '암', '진단', '용', '분자영상', '형광', '프로', '브...",0.0
4,"['위암', '환자', '의', '항암제', '반응', '예측', '을', '위',...",0.0
...,...,...
217875,"[포장, 용, 인, 디케이, 팅, 잉크, 제조, 기술, 개발, 및, 인쇄, 공정, ...",NaN
217876,"[루시아, 신, 속, 정확, 저렴한, 현장, 진단, 용, 혁신, 적, 혈액, 진단,...",NaN
217877,"[ITER, 삼중수소, 저장, 공급, DU, 용기, 개발, 및, 시험, ITER, ...",NaN
217878,"[무결, 정립, 계, 금속, 초, 박막, 의, 제조, 및, 이를, 활용, 한, 기초...",NaN


## 3. 모델링 & 학습 및 예측

In [19]:
import gensim
df = final_df
sentences = df['contents'].values
model = gensim.models.Word2Vec(sentences)

In [20]:
words = list(model.wv.vocab)
filename = '벡터.txt'
model.wv.save_word2vec_format(filename, binary=False)

In [21]:
import os

embeddings_index = {}
f = open(os.path.join('', '벡터.txt'),  encoding = "utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embeddings_index[word] = coefs
f.close()

In [75]:
seperation = 174304 
num = int(seperation *0.8)

In [83]:
X_train = df.loc[:seperation , 'contents'].values
y_train = df.loc[:seperation, 'label'].values

#X_val = df.loc[num:seperation , 'contents'].values
#y_val = df.loc[num:seperation , 'label'].values

X_test = df.loc[seperation:, 'contents'].values
y_test = df.loc[seperation:, 'label'].values

total_reviews = df.values
max_length = 200
sentences = final_df['contents'].tolist()

In [99]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

VALIDATION_SPLIT = 0.15

# 벡터화
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(sentences)
sequences = tokenizer_obj.texts_to_sequences(sentences)

# 문장 패딩
word_index = tokenizer_obj.word_index
print('Found %s unique tokens.' % len(word_index))

sentences_pad = pad_sequences(sequences, maxlen=max_length)
labels =  df['label'].values

train_review_pad = sentences_pad[:seperation]
test_review_pad = sentences_pad[seperation:]
train_rating = labels[:seperation]


# train/val set으로 나누기
indices = np.arange(train_review_pad.shape[0])
np.random.shuffle(indices)

train_review_pad = train_review_pad[indices]
Ys = train_rating[indices]
num_validation_samples = int(VALIDATION_SPLIT * train_review_pad.shape[0])

X_train_pad = train_review_pad[:-num_validation_samples]
y_train = Ys[:-num_validation_samples]
X_test_pad = train_review_pad[-num_validation_samples:]
y_test = Ys[-num_validation_samples:]

Found 354848 unique tokens.


In [100]:
EMBEDDING_DIM = 100
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word_index.items():
    if i > num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

print(num_words)

354849


In [101]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Embedding, Flatten, Dropout
from tensorflow.python.keras.layers.convolutional import Conv1D
from tensorflow.python.keras.layers.convolutional import MaxPooling1D
from tensorflow.python.keras.initializers import Constant


model = Sequential()
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=max_length,
                            trainable=False)

model.add(embedding_layer)

model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))

model.add(MaxPooling1D(pool_size=2))

model.add(Dense(128, activation='relu'))

model.add(Dropout(0.3))

model.add(Flatten())

model.add(Dense(46, activation='sigmoid'))


print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 200, 100)          35484900  
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 196, 128)          64128     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 98, 128)           0         
_________________________________________________________________
dense_6 (Dense)              (None, 98, 128)           16512     
_________________________________________________________________
dropout_3 (Dropout)          (None, 98, 128)           0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 12544)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 46)               

In [107]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras.utils import np_utils

Y = np.concatenate([y_train,y_test])
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
y_train = dummy_y[:-num_validation_samples]
y_val = dummy_y[-num_validation_samples:]

In [109]:
import datetime
stime2 = datetime.datetime.now()
print("training start")

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


hist= model.fit(X_train_pad, y_train, batch_size=64, epochs=5, 
                validation_data=(X_test_pad, y_val), verbose=2)

etime2 = datetime.datetime.now()
mtime2 = etime2-stime2

print("training complete")
print("종료시간: ",etime2,"\n소요시간: ",mtime2,'\n\n')

loss, accuracy = model.evaluate(X_test_pad, y_val, batch_size=64)
print('Accuracy: %.2f' % (accuracy*100),'%')

training start
Train on 148159 samples, validate on 26145 samples
Epoch 1/5
148159/148159 - 193s - loss: 2.9755 - accuracy: 0.8179 - val_loss: 2.1773 - val_accuracy: 0.8183
Epoch 2/5
148159/148159 - 185s - loss: 1.6562 - accuracy: 0.8179 - val_loss: 1.3027 - val_accuracy: 0.8183
Epoch 3/5
148159/148159 - 191s - loss: 1.1781 - accuracy: 0.8179 - val_loss: 1.1058 - val_accuracy: 0.8183
Epoch 4/5
148159/148159 - 191s - loss: 1.0862 - accuracy: 0.8179 - val_loss: 1.0729 - val_accuracy: 0.8183
Epoch 5/5
148159/148159 - 194s - loss: 1.0709 - accuracy: 0.8179 - val_loss: 1.0670 - val_accuracy: 0.8183
training complete
종료시간:  2021-07-10 16:28:16.106839 
소요시간:  0:15:53.584477 


26145/26145 [==============================] - 11s 435us/sample - loss: 1.0670 - accuracy: 0.8183
Accuracy: 81.83 %


In [40]:
model.save('model1')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model1\assets


In [41]:
tqdm.pandas()
test['contents'] = test['data'].apply(lambda x : text_preprocess(x))
test['contents'] = test['contents'].progress_apply(lambda x: tokenize(x))
test

100%|██████████████████████████████████████████████████████████████████████████| 43576/43576 [1:13:36<00:00,  9.87it/s]


,과제명,요약문_연구내용,data,contents
0,R-FSSW 기술 적용 경량 차체 부품 개발 및 품질 평가를 위한 64채널 C-SC...,○ 1차년도\n\n . 개발 탐촉 시스템의 성능 평가 위한 표준 시편 제작 시...,R-FSSW 기술 적용 경량 차체 부품 개발 및 품질 평가를 위한 64채널 C-SC...,"[RFSSW, 기술, 적용, 경량, 차체, 부품, 개발, 및, 품질, 평가, 를, ..."
1,다입자계를 묘사하는 편미분방정식에 대한 연구,연구과제1. 무한입자계의 동역학 / 작용소(operator) 방정식에 대한 연구\n...,다입자계를 묘사하는 편미분방정식에 대한 연구연구과제1. 무한입자계의 동역학 / 작용...,"[다, 입자, 계, 를, 묘사, 하는, 편미분방정식, 에, 대한, 연, 구, 연, ..."
2,저출생체중아 가족을 위한 지역사회중심의 퇴원후 추후관리프로그램 개발,본 연구는 퇴원 후 저출생체중아의 퇴원 후 추후관리 프로그램 중 가정방문 모델과 가...,저출생체중아 가족을 위한 지역사회중심의 퇴원후 추후관리프로그램 개발본 연구는 퇴원 ...,"[저, 출생, 체중, 아, 가족, 을, 위, 한, 지역, 사회, 중심, 의, 퇴원,..."
3,폴리싱용 5축 CNC 정밀 마이크로 시스템 개발,(1) 0.5~1㎛의 가공정밀도(면조도) 구현\n\n\n - 국내에서는 연삭기를 제...,폴리싱용 5축 CNC 정밀 마이크로 시스템 개발(1) 0.5~1㎛의 가공정밀도(면조...,"[폴리, 싱용, 5, 축, CNC, 정밀, 마이크로, 시스템, 개발, 1, 051,..."
4,다결정재료의 미세조직구조를 고려한 확률론적 응력부식균열 생성예측모델 개발,"총 연구기간은 3년으로, 연차별 연구내용 및 범위는 다음과 같다.\n\n ∙ 1차...",다결정재료의 미세조직구조를 고려한 확률론적 응력부식균열 생성예측모델 개발총 연구기간...,"[다, 결정, 재료, 의, 미세, 조, 직구, 조, 를, 고려, 한, 확률론, 적,..."
...,...,...,...,...
43571,포장용 인디케이팅 잉크 제조기술 개발 및 인쇄 공정 기술 개발,■ 포장용 인티케이팅 잉크의 신뢰성 확보 \n\n\n■ 포장용 인티케이팅 잉크의 양...,포장용 인디케이팅 잉크 제조기술 개발 및 인쇄 공정 기술 개발■ 포장용 인티케이팅 ...,"[포장, 용, 인, 디케이, 팅, 잉크, 제조, 기술, 개발, 및, 인쇄, 공정, ..."
43572,루시아-신속/정확/저렴한 현장진단용 혁신적 혈액진단 개발,1) 본 연구팀이 보유하고 있는 특허기술인 C-ECL은 전세계에 존재하는 두 개의 ...,루시아-신속/정확/저렴한 현장진단용 혁신적 혈액진단 개발1) 본 연구팀이 보유하고 ...,"[루시아, 신, 속, 정확, 저렴한, 현장, 진단, 용, 혁신, 적, 혈액, 진단,..."
43573,ITER 삼중수소 저장·공급 DU용기 개발 및 시험,ITER SDS DU bed의 개발 제작 및 삼중수소 저장?공급 및 in-bed c...,ITER 삼중수소 저장·공급 DU용기 개발 및 시험ITER SDS DU bed의 개...,"[ITER, 삼중수소, 저장, 공급, DU, 용기, 개발, 및, 시험, ITER, ..."
43574,무결정립계 금속 초박막의 제조 및 이를 활용한 기초 연구,본 연구실의 연구는 크게 다음과 세 영역으로 구성되어 있음. \n금속 초박막 합성 ...,무결정립계 금속 초박막의 제조 및 이를 활용한 기초 연구본 연구실의 연구는 크게 다...,"[무결, 정립, 계, 금속, 초, 박막, 의, 제조, 및, 이를, 활용, 한, 기초..."


In [110]:
results = model.predict(test_review_pad)
results=tf.argmax(results, axis=1)
sample_submission['label']=results
sample_submission

,index,label
0,174304,0
1,174305,0
2,174306,0
3,174307,0
4,174308,0
...,...,...
43571,217875,0
43572,217876,0
43573,217877,0
43574,217878,0


In [112]:
sample_submission.to_csv('sub.csv', index=False)

In [111]:
sample_submission['label'].value_counts()

0     43574
38        1
23        1
Name: label, dtype: int64

# --------------------------------------------------------
## BERT 모델: 메모리 부족으로 실행불가

In [8]:
new_train = list(zip(train.data, train.label))

In [39]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [42]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import torch.nn.functional as F

In [45]:
optimizer = Adam(model.parameters(), lr=1e-6)

itr = 1
p_itr = 500
epochs = 1
total_loss = 0
total_len = 0
total_correct = 0


model.train()
for epoch in range(epochs):
    
    for text, label in new_train:
        optimizer.zero_grad()
        
        # encoding and zero padding
        encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
        padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
        
        sample = torch.tensor(padded_list)
        labels = torch.tensor(label)
        outputs = model(sample, labels=labels)
        loss, logits = outputs

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        print(itr)
        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0

        itr+=1

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:79] data. DefaultCPUAllocator: not enough memory: you tried to allocate 2642411520 bytes.

# ------------------------------------------------------------------------------------------------

In [ ]:
results = cls_model.predict(test_inputs)
results=tf.argmax(results, axis=1)

In [ ]:
sample_submission['label']=results

In [ ]:
sample_submission

In [ ]:
sample_submission.to_csv('bert_baseline.csv', index=False)